In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 29 10:50:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from __future__ import print_function
import keras,tensorflow
from keras.datasets import mnist
from keras.models import Sequential , Model
from keras.layers import Input ,  Dense, Dropout, Flatten , Activation , ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D  , Add , AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from keras import backend as K
from keras.regularizers import l2
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform


In [ ]:
import numpy as np
x_train = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/sum/512/X.npy')
y_train = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/sum/512/y.npy')

print(y_train.shape)
print(x_train.shape)
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=5,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.25, 
    rescale=1./4095,)
datagen.fit(x_train)


(2800, 4)
(2800, 512, 512, 1)


In [ ]:
import numpy as np
x_test = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/test/512/X.npy')
y_test = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/test/512/y.npy')
x_test = x_test/4095
print(x_test.shape)
print(y_test.shape)

(835, 512, 512, 1)
(835, 4)


proposed_model_resnet50

In [ ]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def ResNet50(input_shape=()):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    #X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(64, (5, 5), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    #X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    #X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
#proposed
base_model = ResNet50(input_shape=(128, 128, 1))
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='tanh', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(512, activation='tanh', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense(4,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)
model = Model(inputs=base_model.input, outputs=headModel)

model.compile(optimizer='sgd', loss="binary_crossentropy", metrics=['binary_accuracy'])

#model.summary()

In [ ]:
#proposed
base_model = ResNet50(input_shape=(512, 512, 1))
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(512, activation='tanh', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(256, activation='tanh', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense(4,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)
model = Model(inputs=base_model.input, outputs=headModel)

model.compile(optimizer='sgd', loss="binary_crossentropy", metrics=['binary_accuracy'])

#model.summary()

In [ ]:
import tensorflow
cb_res = tensorflow.keras.callbacks.ModelCheckpoint(
    '/content/RES_512_proposed.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=16,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb_res, 
         epochs=50)
print("Total time: ", time.time() - start, "seconds")

In [ ]:
acc = model1.history['binary_accuracy']                                                                                                         
val_acc =model1.history['val_binary_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

In [ ]:
from tensorflow import keras
loadedModel = keras.models.load_model('/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/softmax/RESNET50_512_standard.h5')
loadedModel.evaluate(x_test,y_test,batch_size = 10)

In [ ]:
y_pre = loadedModel.predict(x_test)
y_pre = np.round(y_pre)
y_pre
y_true = y_test
from sklearn.metrics import classification_report
import numpy as np

labels =['A', 'B','C','D']
print(classification_report(y_true, y_pre, target_names=labels))
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(np.asarray(y_test).argmax(axis=1), np.asarray(y_pre).argmax(axis=1))

#confusion_matrix(y_true,y_pre)
sns.heatmap(cm,cmap="Blues", annot = True,annot_kws={"size": 16},fmt='g')


standard_model_resnet50

In [ ]:
cb_res1 = tensorflow.keras.callbacks.ModelCheckpoint(
    '/content/RES_512.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights = None, include_top=True, input_shape=(128,128,1))
for layer in base_model.layers[:]:
    layer.trainable = True
#base_model.summary()
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(base_model)
 
# Add new layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
#model.compile(optimizer='sgd', loss="binary_crossentropy",metrics=['binary_accuracy'])

opt = SGD( momentum=0.9)
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=['binary_accuracy'])
# Show a summary of the model. Check the number of trainable parameters
model.summary()

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=32,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb_res1, 
         epochs=50)
print("Total time: ", time.time() - start, "seconds")

In [ ]:
acc = model1.history['binary_accuracy']                                                                                                         
val_acc =model1.history['val_binary_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

In [ ]:
from tensorflow import keras
loadedModel = keras.models.load_model('/content/RES_224.h5')
loadedModel.evaluate(x_test,y_test)